# Operational Snow Water Equivalent (SWE) Products 
In this lab we will look at operational SWE products. These are standard products, produced in near real time (NRT), that are publically avalible. A word of cacution: as you might remember from the lectures, estimating SWE in the mountains is hard. Operational SWE products perform better in large, flat areas with consisten depth and densisty of snowpack across many square kilometeres. Fortunatly there is a lot of development in this space and some of the products are consistently improving. So these methods may be useful for future, better datasets as well. 

## Two types of SWE products: Measurements and Models
There are two main catagories of operational SWE products. Measurements of SWE and process based model estiamtes of SWE. We will explore an example of both. For Measurement of SWE, we will look at the estimate from the unified AMSR SWE products which provide daily estames of global SWE (albet with some caviats). For process based model estaimtes of SWE we will explore the NRT ERA-5 SWE estimates from ECMWF. 

Goals of the lab:
- Gain familiarity with SWE plots.
- Understand the two main sources of SWE estimates: measurements and models
- Gain familiarty with data interpolation
- Data sceince techniques to visualize the SWE in the context of historical records

# PART 1: AMSR-Unified SWE estimates

The first product we are going to look at is the AMSR unifed SWE product. These data are collected from satellies that measure the emission of microwave radiation from earth. Snow scatters and reduces this signal, so based on the measured reduction in passive microwave signal, we can estimate the snow water equivalent. This method works best in areas that have shallow snowpack and are flat for hundreds of kilometers. 

This AMSR Unified SWE product is produced using data collected by the Advanced Microwave Scanning Radiometer (AMSR-E) on board the Aqua satellite from June 2002 to October 2011, and from the Advanced Microwave Scanning Radiometer 2 (AMSR2) on board the JAXA GCOM-W1 satellite from July 2012 to the present. The purpose of the AMSR Unified Data Set is to provide the science community with intercalibrated climate products from both the AMSR-E and the AMSR2 instruments. 

The data sets are archived here:

AMSR-E SWE: 2002-2011
https://nsidc.org/data/AE_DySno/versions/2


AMSR2 SWE: 2012-now 
https://nsidc.org/data/AU_DySno/versions/1


These data are avalible as daily, 5-day maximum, and monthly maximum datasets. We will look at the daily data in this lab excersize. 
## Set up the Lab workspace
To explore these datasets we need to import the require packages for geospatial analysis in python

In [ ]:
import geopandas as gpd 
import rasterio as rio
from rasterio.plot import show
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import os
import contextily as ctx
%matplotlib inline

## Choose an Area of Interest
We have three different AOIs. Choose one of the following to complete your lab: Langtang catchment in Nepal, Panjsher basin in Afganistan, or wangchu watershed in Bhutan. 

In [ ]:
#lets look at the Area of Interest (AOI) on a map. Pick one of the following:

#choose on of the following areas for your labratory. 
#Feel free to explore each of them!
#panjsher_basin_afghanistan
#wangchu_watershed_bhutan
#langtang_catchment_nepal
############## INSERT AOI WATERSHED NAME HERE ###################
watershedName='panjsher_basin_afghanistan'
#################################################################
aoiFile=watershedName+'_wgs84.shp'
basinMaskFile=watershedName+'_basinMask.tif'
aoi_AMSR_SWE=watershedName+'_AMSR_SWE.tif'


## Plot the Area of Interest on a basemap

In [ ]:
#load the shapefile with geopandas and display on a basemap with contextily
#The 'EPSG' codes are standerdized information that discribe which projection the datasets 
#are in. We want to add basemap in same projection as the SWE data and AOI data

#load
AOI = gpd.read_file(aoiFile)
AOI.crs= "EPSG:4326" #({'init': 'epsg:4326'})

#plot
ax = AOI.plot(edgecolor="purple", facecolor="None")
ctx.add_basemap(ax, crs="EPSG:4326"); 

## Visualize the AMSR-E SWE data

Fist, it is importnat to see how the data are delivered. The datasets are delivered for the entire northern hemisphere as a single image each day in the daily dataset. This is not very useful for looking a a small basin! Lets see what the full dataset look like for one day.

In [ ]:
#february 18, 2010 Northerm hemisphere AMSR product.
#- FIX: the AMSR-2 data is .he5 not .hdf 
#print one day of the full AMSR-e data. need to make still.

interesting? This is a unique projection. More information is here:
https://nsidc.org/ease/ease-grid-projection-gt

add some more information about the data storage format for student info



## Explore AMSR SWE for our Area of Interest

We have reprojected, cropped, and assmebeled data cubes of the full AMSR Unifed SWE records for this lab for each area of interest. 

A data cube is a stack of data, for earth science datasets, one common type of datacube is a "time/space" cube. This is when we have data for a specific area, for many days, all assembled into a single file for analysis. For this lab we will be working with time/space datacubs of AMSR-U SWE data. 

to start lets load up the datacube, and visualize it in a few days to get a since of the SWE data that we have to work with. 
## Load SWE datacube and check size

In [ ]:
#load the basin mask
bMask=rio.open(basinMaskFile)
basinMask = bMask.read()
basinMask=np.squeeze(basinMask)

#load the SWE dataset
aoi_AMSR_SWE=watershedName+'_AMSR_SWE.tif'
SWE=rio.open(aoi_AMSR_SWE)
SWEarray = SWE.read()
numel=SWEarray.shape


print(numel)

Now we have loaded the SWE datacube for our study area. the ***'.shape'*** command lets us see the size of our datacube. From this, can you figure out how many days of data are in the datacube? what about how many measurements per day?

## Plot SWE at one location for entire data record
Lets load up the dates for these data, which are stored seperatly and plot one of the pixels in the datacube for the whole record of data.

In [ ]:
dateFile=watershedName+'AMSR_sweDates.csv'
SWEdf=pd.read_csv(dateFile,header=None,names=['sweDates'])
SWEdf['SWE_mm']=pd.DataFrame(SWEarray[:,4,5])#,columns=['SWE_mm'],index=)
SWEdf['sweDates']=pd.to_datetime(SWEdf['sweDates'],format='%Y%m%d')
SWEdf=SWEdf.set_index('sweDates')

#values above 240 are various flags for no data, we will set to NaN
SWEdf[SWEdf>240]=np.nan

Now that we have the data organized into a DataFrame, lets plot one of the pixels in the datacube for the whole record of data.

In [ ]:
# Create figure and plot space
fig, ax = plt.subplots(figsize=(10, 10))

# Add x-axis and y-axis
ax.plot(SWEdf.index.values,
        SWEdf['SWE_mm'],
        color='purple')

# Set title and labels for axes
ax.set(xlabel="Date",
       ylabel="Snow Water Equivalent (mm)",
       title="Daily Snow Water Equivalent \n" + watershedName)

# Rotate tick marks on x-axis
plt.setp(ax.get_xticklabels(), rotation=45)
plt.show()

## Explore one water year of data
Lets look closey at one water year of data, the 2004-2005 water year. 

In [ ]:
WY_04_05=SWEdf['2004-10-01':'2005-09-30']

In [ ]:
# Create figure and plot space
fig, ax = plt.subplots(figsize=(10, 10))

# Add x-axis and y-axis
ax.plot(WY_04_05.index.values,
        WY_04_05,
        color='purple')

# Set title and labels for axes
ax.set(xlabel="Date",
       ylabel="Snow Water Equivalent (mm)",
       title="Daily Snow Water Equivalent \n" + watershedName)

# Rotate tick marks on x-axis
plt.setp(ax.get_xticklabels(), rotation=45)
plt.show()

anything look weird? Why does the SWE disapear on lots of days within the dataseieres?


This is becasue the satellite does not take a measurement everyday. Sometimes the pixel is not images by the satellite, so there are fill values in the datasets on those days. What can we do about this? one solution is to interpolate the value based on measurements on the preceeding days and following days that are valid measurements. With remote sensing data of snow, this is very common. Many times there are numerious days where we just do not have a good measurement. This can be becuase of cloud cover, no picture was taken on that day, or the picture is of poor quality for a variaety of reaseons. (off nadir, noise, ect....)

## Fill and smooth AMSR SWE datacube
Now we will smooth the data. This is an example of a simle way to smooth the data. In the lectures, Karl will show you some of our more advanced routines and we will see the quality of the research grade SWE products that include more difficult time/space smoothing. For now, we will explore how smoothing works with some simpler methods. 

In [ ]:
WY_04_05.head()

In [ ]:
# remake the graph with a cubic spline interpolation of the point estimates from AMSR and then use a rolling average

#calculate the cubic spline estimate of the AMSR data 
amsrSWE_interp=WY_04_05['SWE_mm'].interpolate(method='cubicspline')

# 7 day  moving average of the interpolated data
amsrSWE_avg=amsrSWE_interp.rolling(7).mean()


#plot the smoothed output.
# Create figure and plot space
fig, ax = plt.subplots(figsize=(10, 10))

# Add x-axis and y-axis
ax.plot(amsrSWE_avg.index.values,
        amsrSWE_avg,
        color='purple')

# Set title and labels for axes
ax.set(xlabel="Date",
       ylabel="Snow Water Equivalent (mm)",
       title="Daily Snow Water Equivalent \n" + watershedName)

# Rotate tick marks on x-axis
plt.setp(ax.get_xticklabels(), rotation=45)
plt.show()



In [ ]:
#ADD have students pick a water year 
#and copy/paste/edit the code they need to smooth another water year of SWE data in to the cells below. 

## Calculate volume of SWE in the basin each day

In [ ]:
def compute_volume(dataCube,basinMask):
    idx=np.nonzero(basinMask)
    numValid=np.count_nonzero(basinMask)
    numel=dataCube.shape
    output=np.zeros(numel[0])   
    for i in range(numel[0]):
        daySWE=np.squeeze(dataCube[i,:,:])
        
        #remeber that values above 240 are fill in the original dataset, so we dont want to add - assume missing data
        daySWE[daySWE>240]=0
        
        #volume of water in each pixel
        #SWE*area=volumne of water - we want volume in cubic kilometers
        swe2km3=(31**2)/1000000
        daySWE=daySWE*swe2km3
        #volume of water in the basin
        #all data for this lab has been reporjected to 31km pixels equivalent to the ERA-5 grid.
        #AMSR-U SWE is in 25km native resolution      
        output[i]=daySWE[idx].sum()
    return output

dailyBasinSnowWaterVolume=compute_volume(SWEarray,basinMask)
dailyBasinSnowWaterVolume.shape

Lets make a new dataframe with the new volme data included 

In [ ]:
dateFile=watershedName+'AMSR_sweDates.csv'
SWEdf=pd.read_csv(dateFile,header=None,names=['sweDates'])
SWEdf['km3 snowpack storage']=pd.DataFrame(dailyBasinSnowWaterVolume)
SWEdf['sweDates']=pd.to_datetime(SWEdf['sweDates'],format='%Y%m%d')
SWEdf=SWEdf.set_index('sweDates')
SWEdf.head()

# Create figure and plot space
fig, ax = plt.subplots(figsize=(10, 10))

# Add x-axis and y-axis
ax.plot(SWEdf.index.values,
        SWEdf['km3 snowpack storage'],
        color='purple')

# Set title and labels for axes
ax.set(xlabel="Date",
       ylabel="water volume in snow (km3)",
       title="Daily water stored in the snowpack in the basin \n" + watershedName)

# Rotate tick marks on x-axis
plt.setp(ax.get_xticklabels(), rotation=45)
plt.show()

lets look at basin wide water volume for the 2004-2005 water year again

In [ ]:
WY_04_05=SWEdf['2004-10-01':'2005-09-30']
# Create figure and plot space
fig, ax = plt.subplots(figsize=(10, 10))

# Add x-axis and y-axis
ax.plot(WY_04_05.index.values,
        WY_04_05,
        color='purple')

# Set title and labels for axes
ax.set(xlabel="Date",
       ylabel="Snow Water Equivalent (km3)",
       title="Daily Snow Water Equivalent \n" + watershedName)

# Rotate tick marks on x-axis
plt.setp(ax.get_xticklabels(), rotation=45)
plt.show()

Here we can really see those fill values coming into our data. lets smooth these also and see what it looks like

In [ ]:

#calculate the cubic spline estimate of the AMSR data 
amsrSWE_interp=WY_04_05['km3 snowpack storage'].interpolate(method='cubicspline')

# 7 day  moving average of the interpolated data
amsrSWE_avg=amsrSWE_interp.rolling(7).mean()


#plot the smoothed output.
# Create figure and plot space
fig, ax = plt.subplots(figsize=(10, 10))

# Add x-axis and y-axis
ax.plot(amsrSWE_avg.index.values,
        amsrSWE_avg,
        color='purple')

# Set title and labels for axes
ax.set(xlabel="Date",
       ylabel="Snow Water Equivalent (km3)",
       title="Daily Snow Water Equivalent \n" + watershedName)

# Rotate tick marks on x-axis
plt.setp(ax.get_xticklabels(), rotation=45)
plt.show()


Important to remeber:
swe is not streamflow.
lots can happen to the water in the environmetn before it gets to the stream
these are estimates. operationapl products are realy poor over mountainous terrain.
more importnat than actual values are the tols and techniques that we can use to work with the data and get information from int (i.e SWE to water volume) 


# PART 2: ERA-5 SWE
lets look at the ERA-5 data now. 

add url to era-5 data, add intro text

explain it will be the same operaiton as we did with the AMSR data




In [ ]:
#clean up our workspace we dont need to keep some of the data above loaded anymore


## load ERA-5

In [ ]:
aoi_ERA5_SWE=watershedName+'_ERA5_SWE.tif'
SWE=rio.open(aoi_ERA5_SWE)
SWEarray = SWE.read()
numel=SWEarray.shape
print(numel)

data frame of era5 + dates for one pixel (same pixel as the AMSR part of the lab

In [ ]:
dateFile=watershedName+'ERA5_sweDates.csv'
SWEdf=pd.read_csv(dateFile,header=None,names=['sweDates'])
SWEdf['SWE_mm']=pd.DataFrame(SWEarray[:,4,5])#,columns=['SWE_mm'],index=)
SWEdf['sweDates']=pd.to_datetime(SWEdf['sweDates'],format='%d-%b-%Y %H:%M:%S')
SWEdf=SWEdf.set_index('sweDates')
SWEdf.head()

In [ ]:
# Create figure and plot space
fig, ax = plt.subplots(figsize=(15, 10))

# Add x-axis and y-axis
ax.plot(SWEdf.index.values,
        SWEdf['SWE_mm'],
        color='purple')

# Set title and labels for axes
ax.set(xlabel="Date",
       ylabel="Snow Water Equivalent (mm)",
       title="Daily Snow Water Equivalent \n" + watershedName)

# Rotate tick marks on x-axis
plt.setp(ax.get_xticklabels(), rotation=45)
plt.show()

Look Different? ERA5 gives a much longer historical record of the SWE for the location. This is a nice feture of the models and a dangerous one. They can calculate values for times when they do not have much data. 

look at 2004-2005


In [ ]:
#add code here
WY_04_05=SWEdf['2004-10-01':'2005-09-30']

# Create figure and plot space
fig, ax = plt.subplots(figsize=(15, 10))

# Add x-axis and y-axis
ax.plot(WY_04_05.index.values,
        WY_04_05,
        color='purple')

# Set title and labels for axes
ax.set(xlabel="Date",
       ylabel="Snow Water Equivalent (mm)",
       title="Daily Snow Water Equivalent \n" + watershedName)

# Rotate tick marks on x-axis
plt.setp(ax.get_xticklabels(), rotation=45)
plt.show()

no need to smooth! one part of the model outputs is that they are continous. the physical equations are used to generate estimates between measurements, this is a form of "data assimmilarion" where the combination of models and measuremtns enable you to make the best estimate possible, considering all sourceig of inforation you have avalible, at any timestep.

Now that we have so many years of data, lets make a plot that lets us contextualize any years data wihtin the historical context for that region

In [ ]:
np.unique(SWEdf.index.year)

In [ ]:
WY_04_05=SWEdf['2004-10-01':'2005-09-30']


# Create figure and plot space
#fig, ax = plt.subplots(figsize=(10, 10))
plt.figure(figsize=(15,10));
#plot all the background years
yrs=np.unique(SWEdf.index.year)
numYears=len(yrs)-1

for i in range(numYears):
    sDate=str(yrs[i])+'-10-01'
    eDate=str(yrs[i+1])+'-09-30'
    WY=SWEdf[sDate:eDate]


# Add x-axis and y-axis - x axis is not year this tim ebecuase we have lots of years overlaied
    plt.plot(list(range(len(WY))),
            WY,
            color='grey')


# Rotate tick marks on x-axis
#plt.show()
#xData=list(range(len(WY_04_05)))
#now lets highlight the 2004-2005 water year

# Add x-axis and y-axis
plt.plot(list(range(len(WY_04_05))),
         WY_04_05,
         color='purple')

# Set title and labels for axes
titleText='Daily Snow Water Equivalent \n' + watershedName
plt.title(titleText);
plt.xlabel("Water Year Days");
plt.ylabel("Snow Water Equivalent (mm)");
#plt.setp(ax.get_xticklabels(), rotation=45)


This graph gives some context of the highlighted year we are interested in, in the context of other years. But the graph is still kind of messy. Lets look at another way we can visualize a year of interest in the context of the historical record. What is the driest year? The wettest year? To figure this out, and plot them along with some more historical context, we will use on of the more powerful datascience functions in python the "GroupBy" funciton you can use on pandas dataframes. We can use GroupBy to find the maximum SWE of each calendar year, which we can see from the above graph, for this pixels occurs at peak SWE each spring. [add some more info about group by and some links to good online references]  First, lets use GroupBy to see what the peak SWE was each year for this pixel. 

In [ ]:
#use groupby to split the dataframe into tables for each year, and then find the maximum value in the 'SWE_mm' column
peakSWE=SWEdf.groupby(SWEdf.index.year)['SWE_mm'].max().reset_index()
#the data goes into february 2021, so we want to drop the 2020-2021 water year as it is not yet to peak SWE
peakSWE=peakSWE[:-1]
#view results
print(peakSWE)

Now we have a list of the peak SWE each year, we can use this information to identify the water year that had the deepest snowpack and the water year that had the shallowest snowpack for our contextualization graph

In [ ]:
#what was the driest year?
dryYear=peakSWE.loc[peakSWE["SWE_mm"].idxmin()]['sweDates']
print('driest year:', dryYear, 'peak SWE', peakSWE["SWE_mm"].min(), 'mm')

In [ ]:
#what was the wettest year? for this we use .idxmax() instead of .idxmin()
wetYear=peakSWE.loc[peakSWE["SWE_mm"].idxmax()]['sweDates']
print('wettest year:', wetYear, 'peak SWE', peakSWE["SWE_mm"].max(), 'mm')

In [ ]:
numYears=[dryYear, wetYear]
print(numYears)

In [ ]:
#how different are the wettest and the driest year?
plt.figure(figsize=(15,10));

numYears=[dryYear, wetYear]
for i in numYears:
    sDate=str(i-1)+'-10-01'
    eDate=str(i)+'-09-30'
    WY=SWEdf[sDate:eDate]


# Add x-axis and y-axis - x axis is not year this tim ebecuase we have lots of years overlaied
    plt.plot(list(range(len(WY))),
            WY,
            color='grey')

titleText='Daily Snow Water Equivalent \n' + watershedName
plt.title(titleText);
plt.xlabel("Water Year Days");
plt.ylabel("Snow Water Equivalent (mm)");
#plt.setp(ax.get_xticklabels(), rotation=45)


Now that we know the wettest and driest years, lets use that information to add historical context to our graph of the year of interest. In additon, we can look at the interquartile range of SWE through time. THis is the bounds for any given day for which the SWE model is within the range 50% of the time. 

In [ ]:
#find the interquartile range (25th percintile and 75th percintile of SWE from ERA-5 for this pixel 
#for each day of year across the record. 
DOWY=np.arange(366)
SWE_q1=np.zeros(len(DOWY))
SWE_q3=np.zeros(len(DOWY))

for i in range(0,365):
    #get the data for the day of the year for each year in the record
    temp=SWEdf[SWEdf.index.dayofyear==i+1]
    
    #figure out which day of the water year the day of the year is (not accounting for leap year) 
    #- could make this a second step in the plot
    if i>=274:
        k=i-274
    else:
        k=i+(365-274)
  
    #fill in the correct day of water year location in the arrays for ploting
    SWE_q1[k]=np.percentile(temp['SWE_mm'],25,interpolation='midpoint')
    SWE_q3[k]=np.percentile(temp['SWE_mm'],75,interpolation='midpoint')

#plot the wettest year, the driest year, the interquartile range, and the year of interest on a single graph

fig, ax=plt.subplots(figsize=(15,10))
ax.fill_between(DOWY,SWE_q1,SWE_q3)

#add the wet and dry year for context
numYears=[dryYear, wetYear]
for i in numYears:
    sDate=str(i-1)+'-10-01'
    eDate=str(i)+'-09-30'
    WY=SWEdf[sDate:eDate]


# Add x-axis and y-axis - x axis is not year this tim ebecuase we have lots of years overlaied
    ax.plot(list(range(len(WY))),
            WY,
            color='red')

    # Add x-axis and y-axis
plt.plot(list(range(len(WY_04_05))),
         WY_04_05,
         color='black')
    
    
titleText='Daily Snow Water Equivalent \n' + watershedName
plt.title(titleText);
plt.xlabel("Water Year Days");
plt.ylabel("Snow Water Equivalent (mm)");
#plt.setp(ax.get_xticklabels(), rotation=45)

This is interesting, dicuss with fellow students how you would interpret this graph. Is the wettest year always the most swe on a given day for a pixel? Is the driest year always the driest? 
lets now calcuate basin wide storage